# Obtenir les coordonnées GPS de chaque ville

In [1]:
import pandas as pd
import requests
import json

In [ ]:
BASE_URL = "https://nominatim.openstreetmap.org/search?"

list_cities = ["Mont Saint Michel",
"St Malo",
"Bayeux",
"Le Havre",
"Rouen",
"Paris",
"Amiens",
"Lille",
"Strasbourg",
"Chateau du Haut Koenigsbourg",
"Colmar",
"Eguisheim",
"Besancon",
"Dijon",
"Annecy",
"Grenoble",
"Lyon",
"Gorges du Verdon",
"Bormes les Mimosas",
"Cassis",
"Marseille",
"Aix en Provence",
"Avignon",
"Uzes",
"Nimes",
"Aigues Mortes",
"Saintes Maries de la mer",
"Collioure",
"Carcassonne",
"Ariege",
"Toulouse",
"Montauban",
"Biarritz",
"Bayonne",
"La Rochelle"]

headers = {
    'User-Agent': 'kayak_best_destinations/1.0 (YOUR_EMAIL_ADRESS)'
}
#L'API bloque les requêtes si l'on ne donne pas ces infos


In [3]:
#Ici on obtient un dictionnaire avec en clé le nom de la ville et en valeur ses infos
dict_cities = {}

for city in list_cities:
    payload = {'q': city, 'countrycodes': 'fr', 'format': 'json', 'limit': 1}
    response = requests.get(BASE_URL, params=payload, headers=headers)
    dict_cities[city] = response.json()

In [4]:
#On créé un nouveau dictionnaire listant le nom des villes, leur longitude et leur latitude
new_dict = {
    'city': [],
    'longitude': [],
    'latitude': []
}
for i in range(len(dict_cities)):
    new_dict['city'].append(list_cities[i])
    new_dict['longitude'].append(dict_cities[list_cities[i]][0]['lon'])
    new_dict['latitude'].append(dict_cities[list_cities[i]][0]['lat'])

In [5]:
#On transforme ce dictionnaire en dataframe
df_city = pd.DataFrame(new_dict)

# Obtenir des informations sur la météo

In [ ]:
BASE_URL = 'https://api.openweathermap.org/data/3.0/onecall'
OPENWEATHER_API_KEY = 'YOUR_API_KEY' #À obtenir sur: https://openweathermap.org/
dict_city_meteo = {}

for i in range(len(df_city)):
    payload = {'lon': float(df_city['longitude'][i]), 'lat': float(df_city['latitude'][i]), 'appid': OPENWEATHER_API_KEY, 'units': 'metric'}
    response = requests.get(BASE_URL, params=payload)
    dict_city_meteo[df_city['city'][i]] = response.json()

In [8]:
from datetime import datetime
from collections import defaultdict

In [9]:
#Ici on créé un dictionnaire compilant le nom des colonnes que l'on veut ajouter à notre dataframe
#Et leurs valeurs pour chaque ville
colonnes = defaultdict(list)
for city in list_cities:
    for i in range(1, len(dict_city_meteo[city]['daily'])):
        jour = dict_city_meteo[city]['daily'][i]
        date = datetime.fromtimestamp(dict_city_meteo[city]['daily'][i]['dt']).strftime('%Y-%m-%d')
        colonnes[f"temp_day J+{i} {date}"].append(jour['temp']['day'])
        colonnes[f"temp_night J+{i} {date}"].append(jour['temp']['night'])
        colonnes[f"temp_min J+{i} {date}"].append(jour['temp']['min'])
        colonnes[f"temp_max J+{i} {date}"].append(jour['temp']['max'])
        colonnes[f"temp_eve J+{i} {date}"].append(jour['temp']['eve'])
        colonnes[f"temp_morn J+{i} {date}"].append(jour['temp']['morn'])
        colonnes[f"feels_like_day J+{i} {date}"].append(jour['feels_like']['day'])
        colonnes[f"feels_like_night J+{i} {date}"].append(jour['feels_like']['night'])
        colonnes[f"feels_like_eve J+{i} {date}"].append(jour['feels_like']['eve'])
        colonnes[f"feels_like_morn J+{i} {date}"].append(jour['feels_like']['morn'])
        colonnes[f"humidity J+{i} {date}"].append(jour['humidity'])
        colonnes[f"wind_speed J+{i} {date}"].append(jour['wind_speed'])
        colonnes[f"main weather J+{i} {date}"].append(jour['weather'][0]['main'])
        colonnes[f"clouds J+{i} {date}"].append(jour['clouds'])
        colonnes[f"pop J+{i} {date}"].append(jour['pop'])

In [10]:
#On assemble nos deux dataframe et on obtient un dataframe du top 35 des villes de France à visiter,
#avec leur longitude, leur latitude et leurs infos météorologiques pour les 7 prochains jours
df_city = pd.concat([df_city, pd.DataFrame(colonnes)], axis=1)

In [11]:
#On ajoute une id pour chaque ville
df_city['id'] = [i+1 for i in range(len(list_cities))]

# Déterminer la liste des 5 villes où il fera le plus beau les 7 prochains jours

## Les critères

On décide de noter la météo les villes en fonction de quatre critères:
 - La température la journée (20% de la note) pour viser une température agréable, ni trop chaude, ni trop froide.
 - La différence entre la température la journée et la température ressentie la journée (20% de la note) pour prendre en compte l'humidité et le vent qui peuvent fortement détériorer l'expérience malgré une température idéale.
 - La couverture nuageuse (10%) parce qu'une journée trop couverte est moins agréable et peut affecter l'ambiance.
 - La pluie (50%) parce qu'un séjour peut être complètement gâché par la pluie.

## Barème de la température de la journée:

 - Témpérature idéale : entre 18 et 25 degrés : 20 points.
 - +/- 5 degré : 10 points.
 - au-delà : 0 point.

## Barème de la différence entre la température et la température ressentie:

 - Différence idéale : - de 3 degré : 20 points.
 - entre (+/-)3 et (+/)6 degrés : 10 points.
 - au-delà : 0 point.

## Barème de la couverture nuageuse : 

 - 0 à 20% : 20 points.
 - 20 à 40% : 15 points.
 - 40 à 60% : 10 points.
 - 60 à 80% : 5 points.
 - 80 à 100% : 0 point.

## Barème de la pluie:

 - 0% : 20 points.
 - 0 à 10% : 15 points.
 - 10 à 30% : 10 points.
 - 30 à 50% : 5 points.
 - 50 à 100% : 0 point.

## Modifier notre dataset:

In [12]:
#On ne garde que les colonnes utiles
prefixes = ['id', 'city', 'longitude', 'latitude', 'temp_day', 'feels_like_day', 'clouds', 'pop']

cols_keep = [col for col in df_city.columns if any(col.startswith(p) for p in prefixes)]

df_city = df_city[cols_keep]

In [13]:
#On créé des moyennes sur la semaine pour chaque critère
df_city = df_city.copy() #pour éviter tout comportement imprévisible, plus tard.
df_city['temp_day_mean'] = df_city[[col for col in df_city.columns if col.startswith('temp_day')]].mean(axis=1)
df_city['difference_feels_like_day_mean'] = abs(df_city['temp_day_mean'] - df_city[[col for col in df_city.columns if col.startswith('feels_like_day')]].mean(axis=1))
df_city['clouds_mean'] = df_city[[col for col in df_city.columns if col.startswith('clouds')]].mean(axis=1)
df_city['pop_mean'] = df_city[[col for col in df_city.columns if col.startswith('pop')]].mean(axis=1)

In [14]:
#Barème de la température
conditions_temp = [
    (df_city['temp_day_mean']>=18)&(df_city['temp_day_mean']<=25),
    (df_city['temp_day_mean']>=13)&(df_city['temp_day_mean']<18) |
    (df_city['temp_day_mean']>25)&(df_city['temp_day_mean']<=30),
    (df_city['temp_day_mean']<13) |
    (df_city['temp_day_mean']>30)
]

score_temp = [20, 10, 0]

#Barème de la différence entre température et température ressentie
conditions_feels_like = [
    (df_city['difference_feels_like_day_mean']<3),
    (df_city['difference_feels_like_day_mean']>=3)&(df_city['difference_feels_like_day_mean']<6),
    (df_city['difference_feels_like_day_mean']>=6)
]

score_feels_like = [20, 10, 0]

#Barème de la couverture nuageuse
conditions_clouds = [
    (df_city['clouds_mean']>=0)&(df_city['clouds_mean']<20),
    (df_city['clouds_mean']>=20)&(df_city['clouds_mean']<40),
    (df_city['clouds_mean']>=40)&(df_city['clouds_mean']<60),
    (df_city['clouds_mean']>=60)&(df_city['clouds_mean']<80),
    (df_city['clouds_mean']>=80)&(df_city['clouds_mean']<=100)
]

score_clouds = [20, 15, 10, 5, 0]

#Barème de la pluie
conditions_pop = [
    (df_city['pop_mean']==0),
    (df_city['pop_mean']>0)&(df_city['pop_mean']<0.1),
    (df_city['pop_mean']>=0.1)&(df_city['pop_mean']<0.3),
    (df_city['pop_mean']>=0.3)&(df_city['pop_mean']<0.5),
    (df_city['pop_mean']>=0.5)&(df_city['pop_mean']<=1)
]

score_pop = [20, 15, 10, 5, 0]

In [15]:
import numpy as np

#Création de score pour chaque critère
df_city['temp_day_score'] = np.select(conditions_temp, score_temp, default=0)
df_city['feels_like_score'] = np.select(conditions_feels_like, score_feels_like, default=0)
df_city['clouds_score'] = np.select(conditions_clouds, score_clouds, default=0)
df_city['pop_score'] = np.select(conditions_pop, score_pop, default=0)

In [16]:
#Création du score générale pondéré de chaque ville
df_city['global_score'] = (
    df_city['temp_day_score']*0.2
    + df_city['feels_like_score']*0.2
    + df_city['clouds_score']*0.1
    +df_city['pop_score']*0.5
)

In [17]:
#On trie de la meilleure ville à la pire
df_city = df_city.sort_values(
    ['global_score', 'pop_score', 'temp_day_score', 'feels_like_score', 'clouds_score'],
    ascending=[False, False, False, False, False]).reset_index()
df_city.drop('index', axis=1, inplace=True)

In [19]:
df_city['rank'] = ['{}. {}'.format(i+1, df_city['city'][i]) for i in range(len(df_city))]

In [21]:
new_cols_keep = [
    'id',
    'city',
    'longitude',
    'latitude',
    'global_score',
    'rank'
]
df_city_light = df_city[new_cols_keep]

In [22]:
df_top_5_city = df_city_light.head()

In [ ]:
df_city.to_csv('../city_data/df_city.csv')
df_city_light.to_csv('../city_data/df_city_light.csv')
df_top_5_city.to_csv('../city_data/df_top_5_city.csv')